# Describir el dataset seleccionado.

Fue Elegido el dataset de manos de [poker](http://archive.ics.uci.edu/ml/datasets/Poker+Hand). <br>
En el juego de Poker, cada jugador tiene un conjunto de cartas, el jugador que tiene una clase mayor de las cartas es el ganador.<br>
Ese dataset contiene más de un millón de combinaciones de cartas, con sus clases<br>
Los atributos son 10: numero de la carta y su naipe para cada una de las 5 cartas del jugador

# Definir cuál es el objetivo de aplicar técnicas de clasificación.

El clasificador vas a definir la clase de la mano del jugador, así es posible definir si tienes una buena mano para continuar jugando, o si debes salir del juego y esperar la próxima mano.

# Definir qué pre‐procesamiento se realizará a los datos. Justificar. Aplicarlo.

El dataset no está muy bien normalizado, una vez que sus atributos 2, 4, 6, 8, 10 cambian de 1 a 13 y el numero más grande es 1, después 13, 12 y así adelante, mientraslos atributos 1, 3, 5, 7, 9 cambian de 1 a 4 y no tienen peso, solo son clases diferentes. <br>
Entonces para hacer la normalización, vamos a cambiar los atributos para booleans. Debemos tener como inputs si el jugador tiene una de las 52 cartas, y la salida es la clase de la carta, entonces cambiaremos de: <br>

1,10,1,11,1,13,1,12,1,1,9 (el jugador tiene 9, 10, 11, 12, 13 del mismo naipe, y la clase es 9) <br>
para:<br>
1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9<br>
Así el dataset está normalizado

In [6]:
import numpy as np
import csv

def read_dataset(path):
    att = []
    targ = []
    
    with open(path, 'r') as file:
        readCSV = csv.reader(file, delimiter=',')
        for row in readCSV:
            att.append([int(num) for num in row[:-1]])
            targ.append([int(row[-1])])
    
    np_att = np.array(att)
    np_targ = np.array(targ)
    
    return [np_att, np_targ]

def pre_process_dataset(path):
    with open(path, 'r') as file:
        new_lines = []
        new_targs = []
        readCSV = csv.reader(file, delimiter=',')
        
        for row in readCSV:
            att = [int(num) for num in row[:-1]]
            targ = int(row[-1])
            new_att = [0 for i in range(52)]
            
            for i in range(0, 10, 2):
                new_att[(att[i+1] + ((att[i] - 1) * 13)) - 1] = 1
            
            new_lines.append(new_att)
            new_targs.append(targ)
        
        np_att = np.array(new_lines)
        np_targ = np.array(new_targs)
    
    return [np_att, np_targ]

def save_dataset(dataset, path):
    with open(path, 'w', newline='') as file:
        writeCSV = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        total_lines = len(dataset[0])
        perc = 0
        last_perc = -1
        
        for i in range(total_lines):
            perc = i*100.0/total_lines
            
            if(int(perc)%5 == 0 and last_perc != int(perc)):
                print("{}%".format(perc))
                last_perc = int(perc)
            
            line = dataset[0][i].tolist()
            line.append(dataset[1][i])
            writeCSV.writerow(line)
            
path = '../Poker Dataset/poker-hand-full.data'
data = pre_process_dataset(path)

# Utilizar y evaluar los classificadores

Los clasificadores intentados fueran el KNN y SVM, pero por cuenta del tamaño del dataset, solo fue utilizado el KNN. Primeramente fue hecho un teste con los datos para saber cuantos vecinos eran mejores para la precisión. Con el teste de k = 1 hasta k = 114 fue elegido un k = 102, por su precisión más grande. Para la evaluación fue utilizada el K-Fold cross validation con el numero de folds = 5, y la precisión fue 91.74%. El código esta descrito abajo, pero no es recomendable correrlo, por tardar mucho (casi 3 dias):

In [ ]:
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

now = time.time()
knn = KNeighborsClassifier(n_neighbors=102)
scores = cross_val_score(knn, X=data[0], y=data[1].ravel(), scoring='accuracy', cv=5)
then = time.time()
print('It took: {} seconds \nFor k={} \nAccuracy:{}'.format(then - now, 102, scores.mean()))

La salida del código arriba fue:<br>

It took: 215935.45600390434 seconds<br>
For k=102<br>
Accuracy:0.9174934884531509<br>

# Mostrar un ejemplo de cómo los clasificadores pueden ser usados

Con un clasificador asi, un juego puede hacer una recomendación para el jugador con su mano, si tiene buenas chances o no. Evaluando la clase de su mano y cuantas manos ganan de suya o cuantas pierden.